## **Importar Librerias**

In [1]:
import os
import requests

## **Descarga de datos**

Se configuran las rutas necesarias para la gestión de datos y la canalización del proyecto. 

Primero, define la ubicación del almacén de metadatos en la carpeta pipeline y el directorio donde se almacenarán los archivos de datos en bruto en la carpeta data/covertype. Luego, especifica la ruta exacta del archivo de datos de entrenamiento, covertype_train.csv, usando la librería os para construir la ruta. Después, se asegura de que el directorio de datos exista, creándolo si es necesario. Finalmente, verifica si el archivo de entrenamiento no está presente; en ese caso, descarga el conjunto de datos desde un enlace del repositorio de UCI Machine Learning usando la librería requests y guarda el contenido directamente en formato binario. Este proceso asegura que los datos requeridos estén disponibles y organizados para las siguientes etapas del flujo de trabajo.

In [2]:
# Directorio de los archivos de datos en bruto
_data_root = './data/covertype'

# Ruta al archivo de datos de entrenamiento en bruto
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')

# Crear el directorio si no existe
os.makedirs(_data_root, exist_ok=True)

# Verificar si el archivo no existe para descargarlo
if not os.path.isfile(_data_filepath):
    # URL del conjunto de datos (Repositorio UCI Machine Learning)
    url = 'https://docs.google.com/uc?export=download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'    
    # Realizar la solicitud HTTP para descargar el archivo con la opción de transmisión
    r = requests.get(url, allow_redirects=True, stream=True)
    # Abrir el archivo en modo escritura binaria y guardar el contenido directamente
    open(_data_filepath, 'wb').write(r.content)

Este comando utiliza !head, una instrucción propia de sistemas Unix/Linux, para mostrar las primeras líneas del archivo de datos ubicado en la ruta especificada por la variable _data_filepath. En este caso, _data_filepath contiene la dirección del archivo covertype_train.csv, previamente descargado.Esto es útil para inspeccionar rápidamente el contenido del archivo y verificar su estructura, como la presencia de encabezados, el formato de los datos y posibles valores faltantes.

In [3]:
!head {_data_filepath}

Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1
3185,45,7,470,68,883,222,225,138,2187,Commanche,C7201,0
3157,88,29,361,95,1756,248,179,39,2766,Rawah,C7745,1
3152,310,7,0,0,5549,201,235,174,2100,Rawah,C7202,1
3302,119,23,309,-30,806,252,212,74,3537,Neota,C7756,1


In [4]:
import tensorflow as tf

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

2025-02-27 01:39:33.212342: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-27 01:39:33.213239: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-27 01:39:33.217447: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-27 01:39:33.239836: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-27 01:39:33.281608: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

El pipeline se ejecutará desde este notebook. Al trabajar en un entorno de notebook, los componentes del pipeline se ejecutarán manualmente (es decir, el usuario asumirá el rol de orquestador). Para esto, TFX proporciona el Interactive Context, que permite ejecutar cada componente paso a paso e inspeccionar sus resultados.

In [5]:
_pipeline_root = './tfx_pipeline'

In [6]:
# Inicializar el InteractiveContext con un archivo sqlite local.
# Si dejas `_pipeline_root` en blanco, la base de datos se creará en un directorio temporal.
# Se puede ignorar sin problema la advertencia sobre el archivo de configuración faltante.
context = InteractiveContext(pipeline_root=_pipeline_root)

## Ejecuta componentes de TFX de forma interactiva

A partir de este punto, es posible ejecutar la canalización de manera interactiva. A lo largo de las siguientes secciones, se presentará el procedimiento para hacerlo, explorando cada uno de los componentes involucrados.

El proceso de la canalización se inicia con el componente ExampleGen, cuya función es preparar y organizar los datos para su posterior procesamiento. Este componente realiza las siguientes tareas:

* **División de datos:** Separa el conjunto de datos en subconjuntos de entrenamiento y evaluación, utilizando por defecto una proporción de 2/3 para entrenamiento y 1/3 para evaluación.

* **Conversión de formato:** Transforma cada fila de datos al formato tf.train.Example, un protocolo de búfer diseñado específicamente para operaciones de TensorFlow, empleado ampliamente por los componentes de TFX.

* **Almacenamiento optimizado:** Comprime y almacena la colección de datos en el directorio _pipeline_root, facilitando el acceso de otros componentes de la canalización. Los datos se guardan en formato TFRecord, lo que mejora el rendimiento en las operaciones de lectura y escritura dentro de TensorFlow, especialmente en colecciones de gran volumen.

In [7]:
# Instanciar el componente ExampleGen con el conjunto de datos CSV de entrada
example_gen = CsvExampleGen(input_base=_data_root)

Al ejecutar el componente, se genera automáticamente una celda de salida que muestra los resultados de la ejecución. Los metadatos asociados a esta ejecución se registran en la base de datos configurada previamente, lo que facilita el seguimiento y la gestión de las distintas ejecuciones del proyecto. Cabe destacar que, al volver a ejecutar el proceso, el valor del atributo .execution_id se incrementará, reflejando la creación de una nueva instancia de ejecución.

In [8]:
# Executar el componente
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

El número presente en el directorio ./pipeline/CsvExampleGen/examples/{number} representa el ID de ejecución asignado al conjunto de datos correspondiente. Cada vez que se reinicia el kernel de este entorno de trabajo y se ejecutan nuevamente las celdas hasta este punto, se crea una nueva carpeta con un identificador distinto. Esto refleja el mecanismo de control de versiones implementado por TFX, el cual permite realizar un seguimiento detallado de cada ejecución y, en caso necesario, retroceder para analizar una ejecución específica.

Adicionalmente, los datos ingeridos se almacenan en el directorio especificado en el campo uri y se encuentran comprimidos en formato gzip, lo que puede verificarse ejecutando la celda proporcionada a continuación.

In [9]:
# Obtener el objeto de artefacto
artifact = example_gen.outputs['examples'].get()[0]

# Imprimir los nombres de las particiones y la URI del artefacto
print(f'Split names: {artifact.split_names}')
print(f'Artifact URI: {artifact.uri}')

Split names: ["train", "eval"]
Artifact URI: ./tfx_pipeline/CsvExampleGen/examples/1


In [10]:
# Obtener la URI del artefacto de salida que representa los ejemplos de entrenamiento
train_uri = os.path.join(artifact.uri, 'Split-train')

# Ver el contenido de la carpeta de entrenamiento
!ls {train_uri}


data_tfrecord-00000-of-00001.gz


En este entorno, puede ser útil examinar algunos ejemplos de los datos, especialmente durante la fase de experimentación. Dado que la colección de datos se guarda en formato TFRecord, es necesario utilizar métodos compatibles con este tipo de archivo. Para ello, será necesario desempaquetar los ejemplos individuales del archivo TFRecord y formatearlos adecuadamente para su visualización. A continuación, se realizará este proceso en las siguientes celdas.

In [11]:
# Obtener la lista de archivos en el directorio (archivos TFRecord comprimidos)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Crear un `TFRecordDataset` para leer estos archivos
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")


La función get_records() está diseñada para extraer y convertir un número especificado de registros del conjunto de datos almacenado en formato TFRecord, transformándolos en diccionarios de Python para facilitar su inspección y análisis. Utiliza el método take() de TensorFlow para limitar la cantidad de registros leídos y accede a los datos serializados en formato binario mediante la propiedad .numpy() del tensor. 

Posteriormente, emplea la clase tf.train.Example() para interpretar el protocolo de buffer de TensorFlow, deserializando cada registro con el método ParseFromString(). Finalmente, convierte el mensaje deserializado en un diccionario de Python usando la función MessageToDict(), proporcionando una representación legible y estructurada de los datos. Esta implementación permite visualizar y comprender el contenido de los archivos TFRecord de manera eficiente y flexible durante el desarrollo y la experimentación.

In [12]:
# Definir una función auxiliar para obtener ejemplos individuales
def get_records(dataset, num_records):
    '''Extrae registros del conjunto de datos proporcionado.
    Args:
        dataset (TFRecordDataset): Conjunto de datos generado por ExampleGen.
        num_records (int): Número de registros a previsualizar.
    Returns:
        list: Lista de registros convertidos a diccionarios de Python.
    '''
    
    # Inicializar una lista vacía
    records = []
    
    # Usar el método `take()` para especificar la cantidad de registros a extraer
    for tfrecord in dataset.take(num_records):
        
        # Obtener la propiedad numpy del tensor
        serialized_example = tfrecord.numpy()
        
        # Inicializar un objeto `tf.train.Example()` para leer los datos serializados
        example = tf.train.Example()
        
        # Parsear los datos serializados (protocolo de buffer)
        example.ParseFromString(serialized_example)
        
        # Convertir el mensaje del protocolo de buffer a un diccionario de Python
        example_dict = MessageToDict(example)
        
        # Agregar el registro a la lista
        records.append(example_dict)
        
    return records


Se extraen tres registros del conjunto de datos TFRecord y los muestra de una forma estructurada y legible. Primero, utiliza la función get_records() para obtener los registros y almacenarlos en la variable sample_records. Luego, emplea el método pprint() del módulo pprint (Pretty Print) para imprimir los registros en un formato más organizado, lo que facilita la inspección de la estructura y el contenido de los datos. Esta práctica es especialmente útil durante la fase de exploración y depuración en un entorno de notebooks.

In [13]:
# Obtener 3 registros del conjunto de datos
sample_records = get_records(dataset, 3)

# Imprimir la salida de los registros
pp.pprint(sample_records)


[{'features': {'feature': {'Aspect': {'int64List': {'value': ['3']}},
                           'Cover_Type': {'int64List': {'value': ['1']}},
                           'Elevation': {'int64List': {'value': ['2876']}},
                           'Hillshade_3pm': {'int64List': {'value': ['144']}},
                           'Hillshade_9am': {'int64List': {'value': ['192']}},
                           'Hillshade_Noon': {'int64List': {'value': ['202']}},
                           'Horizontal_Distance_To_Fire_Points': {'int64List': {'value': ['1557']}},
                           'Horizontal_Distance_To_Hydrology': {'int64List': {'value': ['485']}},
                           'Horizontal_Distance_To_Roadways': {'int64List': {'value': ['2495']}},
                           'Slope': {'int64List': {'value': ['18']}},
                           'Soil_Type': {'bytesList': {'value': ['Qzc3NTc=']}},
                           'Vertical_Distance_To_Hydrology': {'int64List': {'value': ['71']}},


2025-02-27 01:40:09.886962: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## **Generador de estadísticas**

El componente StatisticsGen permite generar estadísticas sobre el conjunto de datos, facilitando el análisis exploratorio de la información. Este componente utiliza TensorFlow Data Validation (TFDV) en su núcleo.

**StatisticsGen** toma como entrada el conjunto de datos previamente ingerido mediante el componente CsvExampleGen, permitiendo obtener una visión general de las características y distribución de los datos.

In [14]:
# Instanciar StatisticsGen con el conjunto de datos ingerido por ExampleGen
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Ejecutar el componente
context.run(statistics_gen)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [15]:
# Mostrar las estadísticas de salida
context.show(statistics_gen.outputs['statistics'])


## **Esquema**

El componente SchemaGen emplea TensorFlow Data Validation (TFDV) para generar un esquema basado en las estadísticas del conjunto de datos. Este esquema define la estructura esperada de los datos, especificando aspectos como los tipos de características, los valores permitidos y la presencia de datos nulos, lo que facilita la detección de anomalías y la validación de la calidad de los datos.

SchemaGen utiliza como entrada las estadísticas producidas por el componente StatisticsGen, enfocándose de manera predeterminada en el subconjunto de entrenamiento (training split). Este enfoque asegura que el esquema refleje adecuadamente las características y distribución de los datos utilizados para el entrenamiento de modelos.

In [16]:
# Instanciar SchemaGen con el conjunto de datos procesado por StatisticsGen
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Ejecutar el componente
context.run(schema_gen)


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [17]:
# Visualizar el esquema
context.show(schema_gen.outputs['schema'])


,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## **Validador de Ejemplos (ExampleValidator)**

ExampleValidator es un componente diseñado para identificar anomalías en el conjunto de datos, utilizando como referencia el esquema generado en la etapa anterior. Al igual que los componentes previos, se basa en TensorFlow Data Validation (TFDV), proporcionando una solución robusta para la validación de la calidad de los datos.

Este componente recibe como insumos las estadísticas generadas por StatisticsGen y el esquema producido por SchemaGen. De manera predeterminada, ExampleValidator compara las estadísticas de la partición de evaluación (evaluation split) con el esquema correspondiente a la partición de entrenamiento (training split), facilitando la detección de inconsistencias y asegurando la integridad del conjunto de datos.

In [18]:
# Instanciar ExampleValidator con los datos procesados por StatisticsGen y SchemaGen
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

# Ejecutar el componente
context.run(example_validator)


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [19]:
# Visualizar los resultados
context.show(example_validator.outputs['anomalies'])

## **Transformación (Transform)**

El componente Transform es responsable de llevar a cabo la ingeniería de características, preparando tanto el conjunto de datos de entrenamiento como el de inferencia. Su función es muy importante ya que permite garantizar la coherencia en el procesamiento de los datos a lo largo de todo el ciclo de vida del modelo.

Este componente recibe como insumos el conjunto de datos proporcionado por ExampleGen, el esquema definido por SchemaGen y un módulo que contiene la función de preprocesamiento personalizada.

En esta sección, se implementará un ejemplo de código personalizado para el componente Transform. Dado que la canalización requiere cargar este código como un módulo, es necesario almacenarlo en el disco utilizando el comando  %%writefile. Como primer paso, se definirán una serie de constantes que agrupan los atributos del conjunto de datos, de acuerdo con las transformaciones que se aplicarán posteriormente. Este archivo se guardará localmente para ser utilizado en la configuración del componente.

In [20]:
# Establecer el nombre del archivo del módulo de constantes
_census_constants_module_file = 'census_constants.py'


In [21]:
%%writefile {_census_constants_module_file}

# Características categóricas (datos de tipo cadena)
CATEGORICAL_FEATURE_KEYS = ['Wilderness_Area', 'Soil_Type']

# Características numéricas marcadas como continuas
NUMERIC_FEATURE_KEYS = [
    'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
]

# Característica que se puede agrupar en intervalos (por ejemplo, Elevation podría ser una opción)
BUCKET_FEATURE_KEYS = ['Elevation']

# Número de intervalos utilizados por tf.transform para codificar cada característica de tipo bucket
FEATURE_BUCKET_COUNT = {'Elevation': 5}

# Característica que el modelo predecirá
LABEL_KEY = 'Cover_Type'

# Función de utilidad para renombrar la característica transformada
def transformed_name(key):
    return key + '_xf'


Writing census_constants.py


In [22]:
# Establecer el nombre de archivo del módulo de transformación
_census_transform_module_file = 'census_transform.py'

In [23]:
import tensorflow as tf
import tensorflow_transform as tft
import census_constants

# Escribir el archivo del módulo de transformación
%%writefile {_census_transform_module_file}

# Desempaquetar el contenido del módulo de constantes
_NUMERIC_FEATURE_KEYS = census_constants.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = census_constants.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = census_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = census_constants.FEATURE_BUCKET_COUNT
_LABEL_KEY = census_constants.LABEL_KEY
_transformed_name = census_constants.transformed_name

# Definir las transformaciones
def preprocessing_fn(inputs):
    """Función de devolución de llamada de tf.transform para preprocesar las entradas.
    Args:
        inputs: mapa de claves de características a características sin transformar.
    Returns:
        Mapa de claves de características de cadena a operaciones de transformación.
    """
    outputs = {}

    # Escalar estas características al rango [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])
    
    # Agrupar estas características en intervalos
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Convertir cadenas a índices en un vocabulario
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convertir las etiquetas a un índice
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY])

    return outputs


In [24]:
with open(_census_transform_module_file, 'w') as f:
    f.write("""import tensorflow as tf
import tensorflow_transform as tft
import census_constants

_NUMERIC_FEATURE_KEYS = census_constants.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = census_constants.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = census_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = census_constants.FEATURE_BUCKET_COUNT
_LABEL_KEY = census_constants.LABEL_KEY
_transformed_name = census_constants.transformed_name

def preprocessing_fn(inputs):
    outputs = {}

    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(inputs[key])
    
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(inputs[key], _FEATURE_BUCKET_COUNT[key])

    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY])

    return outputs
""")
print(f"Archivo {_census_transform_module_file} escrito correctamente.")

Archivo census_transform.py escrito correctamente.


In [25]:
# Ignorar mensajes de advertencia de TensorFlow
tf.get_logger().setLevel('ERROR')

# Instanciar el componente de transformación
transform = Transform(
    examples=example_gen.outputs['examples'],  # Conjunto de ejemplos generados como entrada
    schema=schema_gen.outputs['schema'],      # Esquema generado para los datos
    module_file=os.path.abspath(_census_transform_module_file)  # Archivo del módulo de transformación
)

# Ejecutar el componente de transformación
context.run(transform)


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying census_constants.py -> build/lib
copying census_transform.py -> build/lib
installing to /tmp/tmpjggxtd1g
running install
running install_lib
copying build/lib/census_constants.py -> /tmp/tmpjggxtd1g
copying build/lib/census_transform.py -> /tmp/tmpjggxtd1g
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpjggxtd1g/tfx_user_code_Transform-0.0+f7d999a6a2ca31fd37eb1ae865e4167383bb41f94030a5befd1fe5311766bd72

Processing ./tfx_pipeline/_wheels/tfx_user_code_Transform-0.0+f7d999a6a2ca31fd37eb1ae865e4167383bb41f94030a5befd1fe5311766bd72-py3-none-any.whl


Processing ./tfx_pipeline/_wheels/tfx_user_code_Transform-0.0+f7d999a6a2ca31fd37eb1ae865e4167383bb41f94030a5befd1fe5311766bd72-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Ahora se analizaran los artefactos de salida generados por el componente Transform. Este componente produce varios resultados clave:

* **transform_graph:** Representa el grafo que ejecuta las operaciones de preprocesamiento. Este grafo se incorpora tanto en la fase de entrenamiento como en la de despliegue, asegurando la aplicación coherente de las transformaciones a los datos entrantes.
* **transformed_examples:** Contiene los datos de entrenamiento y evaluación que han sido preprocesados.
* **updated_analyzer_cache:** Almacena los cálculos realizados en ejecuciones previas, optimizando el rendimiento en futuras iteraciones.
Es importante revisar el artefacto transform_graph, el cual apunta a un directorio que alberga tres subdirectorios esenciales para el proceso de transformación.

In [26]:
# Obtener el URI del grafo de transformación
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# Listar los subdirectorios bajo el URI
os.listdir(transform_graph_uri)


['metadata', 'transformed_metadata', 'transform_fn']

El subdirectorio metadata contiene el esquema de los datos originales

* El subdirectorio transformed_metadata almacena el esquema de los datos después del proceso de preprocesamiento.
* El subdirectorio transform_fn incluye el grafo que implementa las transformaciones aplicadas a los datos.

Además, es posible visualizar los tres primeros ejemplos transformados utilizando la función auxiliar definida previamente.

In [27]:
# Obtener el URI del artefacto de salida que representa los ejemplos transformados
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Obtener la lista de archivos en este directorio (todos archivos TFRecord comprimidos)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Crear un `TFRecordDataset` para leer estos archivos
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")


In [28]:
# Obtener 3 registros del conjunto de datos
sample_records_xf = get_records(transformed_dataset, 3)

# Imprimir la salida
pp.pprint(sample_records_xf)


[{'features': {'feature': {'Aspect_xf': {'floatList': {'value': [0.008333334]}},
                           'Cover_Type_xf': {'int64List': {'value': ['0']}},
                           'Elevation_xf': {'int64List': {'value': ['1']}},
                           'Hillshade_3pm_xf': {'floatList': {'value': [0.56916994]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.7559055]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.79527557]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.2172154]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.3540146]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [0.35485706]}},
                           'Slope_xf': {'floatList': {'value': [0.27272728]}},
                           'Soil_Type_xf': {'int64List': {'value': ['3']}},
                           'Verti

2025-02-27 01:41:18.780900: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
